In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/space-images-category/space images/nebula - Google Search/45.jpg
/kaggle/input/space-images-category/space images/nebula - Google Search/56.jpg
/kaggle/input/space-images-category/space images/nebula - Google Search/89.jpg
/kaggle/input/space-images-category/space images/nebula - Google Search/20.jpg
/kaggle/input/space-images-category/space images/nebula - Google Search/58.jpg
/kaggle/input/space-images-category/space images/nebula - Google Search/150.jpg
/kaggle/input/space-images-category/space images/nebula - Google Search/6.jpg
/kaggle/input/space-images-category/space images/nebula - Google Search/109.jpg
/kaggle/input/space-images-category/space images/nebula - Google Search/149.jpg
/kaggle/input/space-images-category/space images/nebula - Google Search/76.jpg
/kaggle/input/space-images-category/space images/nebula - Google Search/71.jpg
/kaggle/input/space-images-category/space images/nebula - Google Search/153.jpg
/kaggle/input/space-images-category/space images/

In [6]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import os
import warnings 

warnings.filterwarnings("ignore")

In [7]:
# Define the path to your dataset
dataset_dir = '/kaggle/input/space-images-category/space images'
os.listdir(dataset_dir)

['nebula - Google Search',
 'stars - Google Search',
 'planets - Google Search',
 'constellation - Google Search',
 'cosmos space - Google Search',
 'galaxies - Google Search']

In [8]:
# Define the categories (labels) for your space images
categories = ['stars', 'galaxies', 'planets', 'cosmos', 'constellation', 'nebula']

In [9]:
# Define the image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 8

In [10]:
# Create an ImageDataGenerator for data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Splitting the dataset into 80% training and 20% testing
)

In [11]:
# Load and preprocess the dataset using the ImageDataGenerator
generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Use the training subset of the dataset
)

Found 887 images belonging to 6 classes.


In [12]:
test_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Use the validation (testing) subset of the dataset
)

Found 220 images belonging to 6 classes.


In [18]:
import keras
from keras.layers import Dense
from keras.applications import DenseNet201
from keras.layers import Flatten
base_model=DenseNet201(include_top=False,input_shape=(img_width, img_height,3))
base_model.trainable=False
model=keras.models.Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(len(categories),activation='softmax'))


74836368/74836368 [==============================] - 1s 0us/step


In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet201 (Functional)    (None, 7, 7, 1920)        18321984  
                                                                 
 flatten_1 (Flatten)         (None, 94080)             0         
                                                                 
 dense_2 (Dense)             (None, 6)                 564486    
                                                                 
Total params: 18,886,470
Trainable params: 564,486
Non-trainable params: 18,321,984
_________________________________________________________________


In [27]:
model_filepath = 'space_model.hdf5'
earlystopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                              mode = 'max' ,
                              patience = 4,
                              verbose = 1)

checkpoint = tf.keras.callbacks.ModelCheckpoint(model_filepath,
                                monitor = 'val_accuracy',
                                mode='max',
                                save_best_only=True,
                                verbose = 1)


callback_list = [earlystopping, checkpoint]



lr_scheduler = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate = 0.01,
    decay_steps = 125,
    decay_rate = 0.8
    )

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_scheduler)

model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics=['accuracy'])

In [28]:
history = model.fit(
    generator,
    batch_size=batch_size,
    validation_data=test_generator,
    verbose=1,
    epochs=30,
    callbacks=[earlystopping, checkpoint]
)

Epoch 1/30
111/111 [==============================] - ETA: 0s - loss: 81.7696 - accuracy: 0.4735
Epoch 1: val_accuracy improved from -inf to 0.57727, saving model to space_model.hdf5
111/111 [==============================] - 60s 440ms/step - loss: 81.7696 - accuracy: 0.4735 - val_loss: 55.1415 - val_accuracy: 0.5773
Epoch 2/30
111/111 [==============================] - ETA: 0s - loss: 34.5486 - accuracy: 0.6877
Epoch 2: val_accuracy did not improve from 0.57727
111/111 [==============================] - 39s 354ms/step - loss: 34.5486 - accuracy: 0.6877 - val_loss: 72.2044 - val_accuracy: 0.5727
Epoch 3/30
111/111 [==============================] - ETA: 0s - loss: 25.0376 - accuracy: 0.7272
Epoch 3: val_accuracy improved from 0.57727 to 0.60000, saving model to space_model.hdf5
111/111 [==============================] - 41s 367ms/step - loss: 25.0376 - accuracy: 0.7272 - val_loss: 51.7258 - val_accuracy: 0.6000
Epoch 4/30
111/111 [==============================] - ETA: 0s - loss: 15.48

In [29]:
# Save the trained model
model.save('space_classification_model.h5')